<a href="https://colab.research.google.com/github/Nik8x/Dask_Python_Dataframe_7GB/blob/master/Dask_Python_Dataframe_7GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Dask provides high-level Array, Bag, and DataFrame collections that mimic NumPy, lists, and Pandas but can operate in parallel on datasets that don’t fit into main memory. Dask’s high-level collections are alternatives to NumPy and Pandas for large datasets.*

[311 Service Requests – 7Gb+ CSV](https://data.cityofnewyork.us/Social-Services/311-Service-Requests/fvrb-kbbt)

[Dask – A better way to work with large CSV files in Python](https://pythondata.com/dask-large-csv-python/)

In [0]:
!wget "https://data.cityofnewyork.us/api/views/fvrb-kbbt/rows.csv?accessType=DOWNLOAD"

In [0]:
!rm -r sample_data/

In [4]:
%%bash
mv rows.csv\?accessType\=DOWNLOAD 311.csv
ls -la

total 8259908
drwxr-xr-x 1 root root       4096 Aug  2 15:04 .
drwxr-xr-x 1 root root       4096 Aug  2 14:27 ..
-rw-r--r-- 1 root root 8458119477 Aug  2 15:03 311.csv
drwxr-xr-x 1 root root       4096 Jul 30 16:17 .config
drwxr-xr-x 2 root root       4096 Aug  2 15:03 .ipynb_checkpoints


In [6]:
%%time
!wc -l 311.csv

21242044 311.csv
CPU times: user 1.71 s, sys: 232 ms, total: 1.94 s
Wall time: 5min 12s


In [0]:
pip install dask[complete]

In [0]:
import dask.dataframe as dd

filename = '311.csv'
df = dd.read_csv(filename, dtype='str')
# the data isn’t read into memory. we’ve just set up the dataframe to be ready to do some 
# compute functions on the data in the csv file using familiar functions from pandas.

In [9]:
df.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,38983109,04/18/2018 07:58:48 PM,06/28/2018 12:00:00 AM,DOB,Department of Buildings,Special Projects Inspection Team (SPIT),Sign/Awning/Marquee - Illegal/No Permit,NaN,11208,2986 FULTON STREET,FULTON STREET,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,NaN,06/28/2018 12:00:00 AM,05 BROOKLYN,BROOKLYN,1016292,187037,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.67999212943248,-73.88447670118943,"(40.67999212943248, -73.88447670118943)"
1,38983146,04/19/2018 09:56:50 AM,04/23/2018 12:00:00 AM,DOB,Department of Buildings,Investigations and Discipline (IAD),Plumbing Work - Unlicensed/Illegal/Improper Wo...,NaN,11435,138-19 JAMAICA AVENUE,JAMAICA AVENUE,NaN,NaN,NaN,NaN,ADDRESS,Jamaica,NaN,NaN,Closed,NaN,04/23/2018 12:00:00 AM,12 QUEENS,QUEENS,1035649,195279,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.70252323470668,-73.81462481863822,"(40.70252323470668, -73.81462481863822)"
2,30715497,05/27/2015 01:12:31 PM,05/28/2015 10:45:00 AM,DOT,Department of Transportation,Street Condition,Pothole,NaN,10012,NaN,NaN,NaN,NaN,CROSBY STREET,SPRING STREET,INTERSECTION,NEW YORK,NaN,NaN,Closed,NaN,05/28/2015 10:45:00 AM,02 MANHATTAN,MANHATTAN,984810,202562,Unspecified,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.72266259352169,-73.99797970010471,"(40.72266259352169, -73.99797970010471)"
3,38983112,04/19/2018 05:10:28 PM,04/20/2018 03:24:30 PM,DOHMH,Department of Health and Mental Hygiene,Indoor Air Quality,Dust from Construction,3+ Family Apartment Building,11215,594 6 STREET,6 STREET,8 AVENUE,PROSPECT PARK WEST,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,06/03/2018 05:10:28 PM,04/20/2018 03:24:30 PM,06 BROOKLYN,BROOKLYN,990744,182290,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.667018292174596,-73.97659128104526,"(40.667018292174596, -73.97659128104526)"
4,30715498,05/28/2015 08:59:23 PM,07/20/2015 03:32:37 PM,HPD,Department of Housing Preservation and Develop...,UNSANITARY CONDITION,PESTS,RESIDENTIAL BUILDING,11216,273 GATES AVENUE,GATES AVENUE,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,NaN,07/20/2015 03:32:37 PM,03 BROOKLYN,BROOKLYN,996389,189054,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.68557800392962,-73.95623074565354,"(40.68557800392962, -73.95623074565354)"


In [11]:
df.columns

Index(['UniqueKey', 'CreatedDate', 'ClosedDate', 'Agency', 'AgencyName',
       'ComplaintType', 'Descriptor', 'LocationType', 'IncidentZip',
       'IncidentAddress', 'StreetName', 'CrossStreet1', 'CrossStreet2',
       'IntersectionStreet1', 'IntersectionStreet2', 'AddressType', 'City',
       'Landmark', 'FacilityType', 'Status', 'DueDate',
       'ResolutionActionUpdatedDate', 'CommunityBoard', 'Borough',
       'XCoordinate(StatePlane)', 'YCoordinate(StatePlane)',
       'ParkFacilityName', 'ParkBorough', 'VehicleType', 'TaxiCompanyBorough',
       'TaxiPickUpLocation', 'BridgeHighwayName', 'BridgeHighwayDirection',
       'RoadRamp', 'BridgeHighwaySegment', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [10]:
# We see that there’s some spaces in the column names. Let’s remove those spaces to make things easier to work with.
%%time
df = df.rename(columns={c: c.replace(' ', '') for c in df.columns})

CPU times: user 17.1 ms, sys: 888 µs, total: 18 ms
Wall time: 17.6 ms


In [15]:
%time df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 38 entries, UniqueKey to Location
dtypes: object(38)CPU times: user 3.71 ms, sys: 125 µs, total: 3.83 ms
Wall time: 3.39 ms


In [16]:
%time df.describe

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 13.4 µs


<bound method _Frame.describe of Dask DataFrame Structure:
                UniqueKey CreatedDate ClosedDate  Agency AgencyName ComplaintType Descriptor LocationType IncidentZip IncidentAddress StreetName CrossStreet1 CrossStreet2 IntersectionStreet1 IntersectionStreet2 AddressType    City Landmark FacilityType  Status DueDate ResolutionActionUpdatedDate CommunityBoard Borough XCoordinate(StatePlane) YCoordinate(StatePlane) ParkFacilityName ParkBorough VehicleType TaxiCompanyBorough TaxiPickUpLocation BridgeHighwayName BridgeHighwayDirection RoadRamp BridgeHighwaySegment Latitude Longitude Location
npartitions=133                                                                                                                                                                                                                                                                                                                                                                                            

In [19]:
%time df.shape

CPU times: user 1.24 ms, sys: 41 µs, total: 1.28 ms
Wall time: 1.25 ms


(Delayed('int-47eef94a-03c8-4dfc-ae4d-a38601b6d4dc'), 38)

In [0]:
%time df.isna().sum().compute()

In [0]:
# create a new dataframe with only 'RADIATOR' service calls
%%time
radiator_df = df[df.Descriptor == 'RADIATOR']

In [16]:
# Let’s see how many rows we have using the ‘count’ command
radiator_df.Descriptor.count()

%%time
# To actually compute the count, you have to call “compute” to get dask to run through the dataframe and count the number of records.
radiator_df.Descriptor.count().compute()

69027

In [0]:
%whos